In [1]:
import secret

In [2]:
import pandas as pd
import numpy as np

In [3]:
from bs4 import BeautifulSoup as bs
import requests
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

import time
import random

In [6]:
# Function to randomize waiting interval between crawling actions (default random range multiplier = [0.95, 2.05])
def randomize_wait_time(base_number):
    return round(random.uniform(base_number*0.95, base_number*2.05), 2)

# Function to randomize waiting interval between crawling actions (allow argument to pass range multiplier)
def randomize_wait_time(base_number, rand_range = [0.95, 2.05]):
    return round(random.uniform(base_number*rand_range[0], base_number*rand_range[1]), 2)

In [7]:
def create_session(user, password):
    # driver = webdriver.Chrome()
    options = Options()
    options.add_argument("--incognito")
    driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options = options)
    login_url = 'https://www.linkedin.com/checkpoint/lg/login?rm_ex=AQHaACq_hiytJQ'
    login_url = 'https://www.linkedin.com/checkpoint/rm/sign-in-another-account'
    # url = 'https://www.linkedin.com/jobs/view/3738522449/?trackingId=3Ooh7LaMkMF4shE6s%2BqArQ%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3D12efa18a...a6430f47%29&midToken=AQHaACq_hiytJQ&midSig=2rNuWr9nBFEaY1&trk=eml-email_job_alert_digest_01-job_card-0-jobcard_body&trkEmail=eml-email_job_alert_digest_01-job_card-0-jobcard_body-null-cwiv5t~lnozuufb~z8-null-null&eid=cwiv5t-lnozuufb-z8&otpToken=MTUwZDFlZTIxMDJjYzljNWIzMjQwNGVjNGYxOWU1YjY4YmM2ZDY0NTliYWE4YjYxNzZjNDA0NjY0ZTVjNWZmMmYyZDRkZjk1MTJiOGYxYzU2ZjlhYTM4ZmU2YjE1MDQ0MjM2M2IwMzYwMTI4MzBiNDk1MWExNywxLDE%3D'

    driver.get(login_url)
    time.sleep(randomize_wait_time(2))

    username_field = driver.find_element(By.ID, 'username')
    username_field.send_keys(user)
    # time.sleep(randomize_wait_time(2))
    password_field = driver.find_element(By.ID, 'password')
    password_field.send_keys(password)
    # time.sleep(randomize_wait_time(2))

    xpath = "//button[text()='Sign in']"
    login_button = driver.find_element(By.XPATH, xpath)
    # login_button = driver.find_element(By.CLASS_NAME, 'btn__primary--large from__button--floating')
    login_button.click()
    # time.sleep(1)
    time.sleep(randomize_wait_time(2))

    cookies = driver.get_cookies()
    driver.quit()
    session = requests.Session()
    for cookie in cookies:
        session.cookies.set(cookie['name'], cookie['value'])

    return session

class PageCrawler:
    def __init__(self):
        self.jobs_page_url = 'https://www.linkedin.com/jobs/'
        self.jobs_page_url = 'https://www.linkedin.com/jobs/search/?keywords=data%20analyst&location=Worldwide'
        # self.emails = 
        # self.passwords = 
        self.sessions = [create_session(secret.user[i], secret.password[i]) for i in range(len(secret.user))]
        self.session_index = 0
        self.headers = [{
            'Authority': 'www.linkedin.com',
            'Method': 'GET',
            'Path': f'{self.jobs_page_url}',
            'Scheme': 'https',
            'Accept': 'application/vnd.linkedin.normalized+json+2.1',
            'Accept-Encoding': 'gzip, deflate, br',
            'Accept-Language': 'en-US,en;q=0.9',
            'Cookie': "; ".join([f"{key}={value}" for key, value in session.cookies.items()]),
            'Csrf-Token': session.cookies.get('JSESSIONID').strip('"'),
            # 'TE': 'Trailers',
            'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/117.0.0.0 Safari/537.36',
            # 'X-Li-Track': '{"clientVersion":"1.12.7990","mpVersion":"1.12.7990","osName":"web","timezoneOffset":-7,"timezone":"America/Los_Angeles","deviceFormFactor":"DESKTOP","mpName":"voyager-web","displayDensity":1,"displayWidth":1920,"displayHeight":1080}'
            'X-Li-Track': '{"clientVersion":"1.13.5589","mpVersion":"1.13.5589","osName":"web","timezoneOffset":-7,"timezone":"America/Los_Angeles","deviceFormFactor":"DESKTOP","mpName":"voyager-web","displayDensity":1,"displayWidth":360,"displayHeight":800}'
        } for session in self.sessions]

    def get_job_id(self):
        return 0

    def get_jobs(self):
        results = self.sessions[self.session_index].get(self.jobs_page_url, headers = self.headers[self.session_index])
        self.session_index = (self.session_index + 1) % len(self.sessions)

        if results.status_code != 200:
            raise Exception('Status code {} for search\nText: {}'.format(results.status_code, results.text))
        results = results.json
        job_ids = {}

        # for r in results['included']:
        #     if r['$type'] == 'com.linkedin.voyager.dash.jobs.JobPostingCard' and 'referenceId' in r:
        #         job_id = int(strip_val(r['jobPostingUrn'], 1))
        #         job_ids[job_id] = {'sponsored': False}
        #         job_ids[job_id]['title'] = r.get('jobPostingTitle')
        #         for x in r['footerItems']:
        #             if x.get('type') == 'PROMOTED':
        #                 job_ids[job_id]['sponsored'] = True
        #                 break
    
        return job_ids



# Get current page url
    curr_url = driver.current_url
    # Get previous page url
    prev_url = curr_url

    jobs_page_url = 'https://www.linkedin.com/jobs/'
    driver.get(jobs_page_url)
    time.sleep(randomize_wait_time(30))

    # Get keywords to search for jobs
    # Insert function to cycle through keywords here

    html_text = driver.page_source
    # driver.quit()
    soup = bs(html_text, 'lxml')

    print(curr_url)
    print(html_text)
    print(soup)

In [ ]:
# Declaring scraping rule to avoid robot detection from LinkedIn
maximum_scraping_limit = 1000 # 1000 pages daily (non-negotiable)
daily_job_pages_to_open_limit = 25

# Setting time intervals
# Pause duration in seconds
pause_duration = 90
pause_randomizer_limit = [1.0, 10.0]

In [2]:
# driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()))
# url = ("https://finviz.com/quote.ashx?t=" + symbol)
# driver.get(url)
# time.sleep(2)

url = 'https://www.linkedin.com/jobs/view/3738522449/?trackingId=3Ooh7LaMkMF4shE6s%2BqArQ%3D%3D&refId=ByteString%28length%3D16%2Cbytes%3D12efa18a...a6430f47%29&midToken=AQHaACq_hiytJQ&midSig=2rNuWr9nBFEaY1&trk=eml-email_job_alert_digest_01-job_card-0-jobcard_body&trkEmail=eml-email_job_alert_digest_01-job_card-0-jobcard_body-null-cwiv5t~lnozuufb~z8-null-null&eid=cwiv5t-lnozuufb-z8&otpToken=MTUwZDFlZTIxMDJjYzljNWIzMjQwNGVjNGYxOWU1YjY4YmM2ZDY0NTliYWE4YjYxNzZjNDA0NjY0ZTVjNWZmMmYyZDRkZjk1MTJiOGYxYzU2ZjlhYTM4ZmU2YjE1MDQ0MjM2M2IwMzYwMTI4MzBiNDk1MWExNywxLDE%3D'
html_text = requests.get(url).text
soup = bs(html_text, 'lxml')

print(html_text)
print(soup)

<code id="i18n_forget_account_error_message" style="display: none;"><!--"Unable to remove account. Try again in sometime."--></code><!DOCTYPE html><html lang="en-US" class="artdeco "><head><meta http-equiv="X-UA-Compatible" content="IE=EDGE"><meta charset="UTF-8"><meta name="viewport" content="width=device-width, initial-scale=1"><meta name="asset-url" id="artdeco/static/images/icons.svg" content="https://static.licdn.com/sc/h/6bja66gymvrvqrp5m6btz3vkz"><title>LinkedIn Login, Sign in | LinkedIn</title><link rel="shortcut icon" href="https://static.licdn.com/sc/h/9lb1g1kp916tat669q9r5g2kz"><link rel="apple-touch-icon" href="https://static.licdn.com/sc/h/1exdo4axa6eaw1jioxh1vu4fj"><link rel="apple-touch-icon-precomposed" href="https://static.licdn.com/sc/h/55ggxxse8uyjdh2x78ht3j40q"><link rel="apple-touch-icon-precomposed" sizes="57x57" href="https://static.licdn.com/sc/h/1exdo4axa6eaw1jioxh1vu4fj"><link rel="apple-touch-icon-precomposed" sizes="144x144" href="https://static.licdn.com/sc

In [ ]:
def get_list_all_job(url = 'https://www.linkedin.com/jobs/'):
    return 0